<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/Lottario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

: 

In [ ]:
df = pd.read_csv("master.csv")

: 

In [ ]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate'], axis=1, inplace=True)

: 

In [ ]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

: 

In [ ]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 7
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


: 

In [ ]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

: 

In [ ]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
#batch_size = 40
batch_size = 100

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(6))
# Adding the last output layer
model.add(Dense(number_of_features))

: 

In [ ]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.95, random_state=95)

# Train the model
history = model.fit(X, y, batch_size=batch_size, epochs=1000, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

: 

In [ ]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

: 

In [ ]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

: 

In [ ]:
''' import datetime
import pytz

# Get the current time in a specific timezone
tz = pytz.timezone('America/New_York')
current_day = datetime.datetime.now(tz)

time_label = "Midday" if (current_day.hour <= 14) else "Evening"

label = current_day.strftime("%Y_%m_%d_") + time_label
model.save(f'Models/{label}.keras')
print("saved model") '''

: 

In [ ]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

: 

In [ ]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(np.unique(scaler.inverse_transform(y_pred).astype(int)[0]))
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[uniq_idx[counts>=2] ]  # <--- Get duplicates

top_5_indices = np.argsort(y_pred[0], axis=0)[::-1][:-10]
top_5_numbers = []

winning_indices = np.where(np.isin(prediction[0], duplicates))[0]

# Print the winning indices
print("The winning indices:", winning_indices)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices:", top_5_indices)
print("The top 5 numbers:", top_5_numbers)

print("The predicted numbers for the lottery game are:  ", temp)
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("winners:", duplicates)
print(len(duplicates)/20)


: 

# Predict the next set of numbers

In [ ]:
next = df.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)

top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_10_numbers = []
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)
print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)


: 

In [ ]:
next = df.copy()
next = next.tail(1)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_next_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:10]
top_5_indices = np.argsort(y_next_pred[0], axis=0)[::-1][:5]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

winning_numbers = []
for index in winning_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    winning_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("winning indices", winning_indices)
print("winning numbers", winning_numbers)
print("\n")

print("The predicted numbers:", prediction)


: 